In [ ]:
import json
import pandas as pd
import glob
import xmltodict, json
from pandasql import sqldf

In [ ]:
pysqldf = lambda q: sqldf(q, globals())

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [ ]:
files = glob.glob('IADownloads/*')

rows = []

for file in files:
    print(file)
    row = []
    
    fname = file.split('/')[1]
    
    if fname != 'Search_IA.ipynb':
    
        with open(file + '/' + file.split('/')[1] + '_meta.xml') as f:

            try:
                jsn = xmltodict.parse(f.read())

                row.append(jsn['metadata']['identifier'])
                row.append(','.join(jsn['metadata']['collection']))
                row.append(jsn['metadata']['mediatype'])
                row.append(jsn['metadata']['title'])
                if 'date' in jsn['metadata']:
                    row.append(jsn['metadata']['date'])
                elif 'year' in jsn['metadata']:
                    row.append(jsn['metadata']['year'])
                else:
                    row.append('')
                if 'description' in jsn['metadata']:
                    row.append(jsn['metadata']['description'])
                else:
                    row.append('')
                row.append(','.join(jsn['metadata']['subject'].split(';')[:-1]))
            except:
                print('error in', file)
        rows.append(row)

In [ ]:
len(rows[0])

In [ ]:
df_metadata = pd.DataFrame(rows, columns=['identifier', 'collection', 'title', 'mediatype', 'year', 'description','subject'])

In [ ]:
df_metadata

In [ ]:
#df_metadata.to_csv('metadata.csv', index=False)

In [ ]:
files = glob.glob('IADownloads/*')

rows = []
for file in files:
    
    fname = file.split('/')[1]
    print(file, fname)
    
    if fname != 'Search_IA.ipynb':
    
        with open(file + '/' + fname + '_files.xml') as f:

            jsn = xmltodict.parse(f.read())
            
            #print(jsn.keys())
            
            for f in jsn['files']['file']:  
                #print(f)
                #print('***', f['@name'].split('.')[-1])
                row = []
                if f['@name'].split('.')[-1] == 'mp4':  
                    dl = f['@name']
                    s = int(f['size'])
                    l = float(f['length'])
                    url = 'https://archive.org/download/' + file.split('/')[1] + '/' + dl
                    row.append(file.split('/')[1])
                    row.append(dl)
                    row.append(s)
                    row.append(l)
                    row.append(url)
                    #print(row)
                    rows.append(row)


In [ ]:
df_files = pd.DataFrame(rows, columns=['identifier', 'name', 'size', 'length', 'url'])

In [ ]:
df = pd.merge(df_files, df_metadata,on='identifier')

In [ ]:
df.to_csv('csv_files/metadata_links.csv', index=False)

In [ ]:
df.head()

In [ ]:
cloud_data_transfer = pysqldf("""
SELECT DISTINCT url FROM df
WHERE url NOT LIKE '%.ia.%'
""")

In [ ]:
cloud_data_transfer.columns = ['TsvHttpData-1.0']

In [ ]:
cloud_data_transfer

In [ ]:
cloud_data_transfer.to_csv('csv_files/cloud_data_transfer.tsv', sep='\t',index=False)